EleutherAI/polyglot-ko-5.8b

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
from peft import PeftModel, PeftConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


In [2]:
!nvidia-smi

Thu Jul 27 10:40:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 50%   40C    P8    24W / 350W |     20MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:4A:00.0 Off |                  N/A |
|  0%   

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-5.8b"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(    # 모델의 성능을 유지하면서 메모리 사용을 최적화하고, 하드웨어 환경에 맞게 데이터를 처리하는 데 도움
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [4]:
from peft import prepare_model_for_kbit_training
# 모델의 훈련 과정에서 필요한 메모리 양을 줄일 수 있습니다.
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [7]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/brianjang7/home1/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: ### 1. 운동 2. 물 3. 영양 섭취 ### Question: With three other ways to live healthy, what is that? SAR D: ### 1. Exercise 2. Water 3. Nutrition Question: Tell us something about your current recovery and you have needed time to get all your muscles up back into shape. SAR D: ### 1. I feel like it's been a little too quickly for me to get back in shape. It's not easy, I know, but simply starting of the exercise and doing it again feels good for me. 2." Because it's not easy, but it takes something too t
